##### Imports

In [1]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### Data Reading and Feature Engineering

In [2]:
from scr.feature_engineering import build_feature_set, \
    process_single_word_inference
from scr.utils import *

import random

# MASK_PROB = 0.5

# Limit the number of words to a smaller number for debugging
word_list = read_words('/home/sayem/Desktop/Hangman/words_250000_train.txt', limit=None)
# word_list = word_list[:100]
# # # Randomly select 1000 words
# # unseen_words = random.sample(word_list, 1000)c

In [3]:
import torch
from scr.feature_engineering import add_features_for_training, \
    calculate_char_frequencies
    
import random
# # Initialize lists for features, labels, missed characters, and original words
# all_features, all_labels, all_missed_chars, original_words = [], [], [], []

char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)
MASK_PROB = 0.5
NGRAM_N = 3

from scr.dataset import HangmanDataset, collate_fn
from torch.utils.data import DataLoader, Dataset

dataset = HangmanDataset(word_list, \
    char_frequency, max_word_length, MASK_PROB, NGRAM_N)


# data_loader = DataLoader(dataset, batch_size=3, \
#     shuffle=True, collate_fn=collate_fn)

# # Example of iterating over the DataLoader
# i = 0
# for i, batch in enumerate(data_loader):
#     inputs, labels, miss_chars, lengths, original_words = batch
    
#     i += 1

#     # print(inputs.shape)
#     # print(labels.shape)
#     # print(missed_chars.shape)
#     # print(lengths.shape)
#     # print(original_words)
#     # print()

#     # break

# print(len(dataset))
# print(i)

In [4]:
# # For inference
from scr.feature_engineering import process_single_word_inference

# Example usage
inference_word = "_p_"
inference_features, inference_missed_chars = \
    process_single_word_inference(inference_word, \
        char_frequency, max_word_length, ngram_n=NGRAM_N)

##### Model Building

In [5]:
# In your main script or Jupyter Notebook
from scr.model import RNN
from scr.feature_engineering import process_single_word_inference, \
    char_to_idx, idx_to_char, calculate_char_frequencies, \
        get_missed_characters

# Configuration for the RNN model
# Configuration for the RNN model
config = {
    'rnn': 'LSTM',
    'vocab_size': 27,  # Assuming 26 letters + 1 for underscore
    'hidden_dim': 512,
    'num_layers': 2,
    'embedding_dim': 200,
    'output_mid_features': 200,
    'miss_linear_dim': 150,
    'dropout': 0.3, 
    'use_embedding': True,
    'lr': 1e-05, 
    'input_feature_size': 5, # Number of features excluding the embedding dimension
    'step_size': 15, 
    'gamma': 0.341,
    'use_cuda': True  # Set to True to use CUDA, False to use CPU
}


In [6]:
from scr.game import guess_character
from scr.game import play_game_with_a_word

# Initialize RNN model
model = RNN(config)
model = model.to(device)

model.save_model('models/model.pth')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # for i in range(1000):
# #     print('Playing', i, 'th game')
# #     success = play_a_game(model, word_to_guess, char_frequency, max_word_length, device)
# #     if success:
# #         print("Won the game!")
# #     else:
# #         print("Lost the game!")
# #     # time.sleep(0.5)  # DO NOT REMOVE

# [total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
# practice_success_rate = total_practice_successes / total_practice_runs
# print('Ran %d practice games out of an allotted 100,000. Practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

In [8]:
word = '_ _ b'
won, final_word, attempts = play_game_with_a_word(model, word, char_frequency, \
    max_word_length, device, max_attempts=6, normalize=True)

Guessed: c, Word: _ _ _ _ _, Attempts remaining: 5
Guessed: t, Word: _ _ _ _ _, Attempts remaining: 4
Guessed: p, Word: _ _ _ _ _, Attempts remaining: 3
Guessed: k, Word: _ _ _ _ _, Attempts remaining: 2
Guessed: i, Word: _ _ _ _ _, Attempts remaining: 1
Guessed: x, Word: _ _ _ _ _, Attempts remaining: 0


In [9]:
won

False

In [10]:
final_word

'_____'

In [13]:
from scr.game import simulate_hangman_games
# Example usage with a word list
word_list = ["___", "python", "openai"]
# num_games = 1000
game_results = simulate_hangman_games(model, char_to_idx, \
    idx_to_char, char_frequency, max_word_length, device, \
        word_list=word_list, num_games=num_games)

___
Guessed: t, Word: _ _ _, Attempts remaining: 5
Guessed: c, Word: _ _ _, Attempts remaining: 4
Guessed: p, Word: _ _ _, Attempts remaining: 3
Guessed: k, Word: _ _ _, Attempts remaining: 2
Guessed: i, Word: _ _ _, Attempts remaining: 1
Guessed: x, Word: _ _ _, Attempts remaining: 0
python
Guessed: t, Word: _ _ t _ _ _, Attempts remaining: 6
Guessed: p, Word: p _ t _ _ _, Attempts remaining: 6
Guessed: c, Word: p _ t _ _ _, Attempts remaining: 5
Guessed: k, Word: p _ t _ _ _, Attempts remaining: 4
Guessed: i, Word: p _ t _ _ _, Attempts remaining: 3
Guessed: f, Word: p _ t _ _ _, Attempts remaining: 2
Guessed: s, Word: p _ t _ _ _, Attempts remaining: 1
Guessed: h, Word: p _ t h _ _, Attempts remaining: 1
Guessed: u, Word: p _ t h _ _, Attempts remaining: 0
openai
Guessed: t, Word: _ _ _ _ _ _, Attempts remaining: 5
Guessed: k, Word: _ _ _ _ _ _, Attempts remaining: 4
Guessed: p, Word: _ p _ _ _ _, Attempts remaining: 4
Guessed: c, Word: _ p _ _ _ _, Attempts remaining: 3
Guessed: i,

In [14]:
game_results

(0.3333333333333333, 6.0, {3: 1.0})

In [ ]:
# current_masked_word = "_i___l__"

# # missed_chars = get_missed_characters(word, char_to_idx)

# predicted_char = predict_next_character(model, current_masked_word, \
#     char_frequency, max_word_length, device=device, use_initial_guess=True)

# # predicted_index = predict_next_character_beam_search(model, current_masked_word, \
# #     missed_chars, \
# #     char_frequency, max_word_length, \
# #     device, normalize=True, beam_width=3)
        
# # predicted_char = idx_to_char[predicted_index]

# predicted_char

##### Playing Games

##### Train

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader

def train_one_epoch(model, data_loader, device=device):
    total_actual_penalty = 0
    total_miss_penalty = 0
    total_batches = 0

    model.train()
    model.to(device)

    for i, batch in enumerate(data_loader):
        inputs, labels, miss_chars, lengths, original_words = batch

        inputs = inputs.to(device)
        labels = labels.to(device)
        miss_chars = miss_chars.to(device)
        # lengths = lengths.to(device)  # Uncomment this if lengths need to be on the device

        outputs = model(inputs, lengths, miss_chars)
       
        actual_penalty, miss_penalty = model.calculate_loss(outputs, labels, \
            lengths, miss_chars, vocab_size=27)

    
        total_actual_penalty += actual_penalty.item()
        total_miss_penalty += miss_penalty.item()
        total_batches += 1

        model.optimizer.zero_grad()
        actual_penalty.backward()
        model.optimizer.step()

    avg_actual_penalty = total_actual_penalty / total_batches if total_batches > 0 else 0
    avg_miss_penalty = total_miss_penalty / total_batches if total_batches > 0 else 0
    return avg_actual_penalty, avg_miss_penalty


    avg_actual_penalty = total_actual_penalty / total_batches if total_batches > 0 else 0
    avg_miss_penalty = total_miss_penalty / total_batches if total_batches > 0 else 0
    return avg_actual_penalty, avg_miss_penalty

In [ ]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

# Split the dataset indices into training and testing sets (80% train, 20% test)
train_indices, test_indices = \
    train_test_split(range(len(dataset)), train_size=0.8, random_state=42)

# Create subsets for training and testing
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

# Create DataLoaders for the training and test datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

num_epochs = 20

for epoch in range(num_epochs):
    # Training step
    avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, train_loader, device)

    print(f"Epoch {epoch+1}: Training data - Avg Actual Penalty: {avg_actual_penalty}, Avg Miss Penalty: {avg_miss_penalty}")

    # Validation step
    
    avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, test_loader, device)
    print(f"Epoch {epoch+1}: Testing data - Avg Actual Penalty: {avg_actual_penalty}, Avg Miss Penalty: {avg_miss_penalty}")

    print()

# win_rate, average_attempts, win_rate_by_length = validate_one_epoch(model, val_loader, \
#     char_to_idx, idx_to_char, char_frequency, max_word_length, device)

In [ ]:
STOP

In [ ]:
from scr.game import simulate_games

In [ ]:
import optuna
from sklearn.model_selection import KFold
import torch
from torch.utils.data import DataLoader, Subset
from scr.model import RNN
from torch.optim.lr_scheduler import StepLR
from scr.utils import *
from scr.early_stopping import EarlyStopping

N_GAMES_PER_EPOCH = 10 ** 3
MAX_ATTEMPTS = 6

def objective(trial, dataset, static_config, num_epochs):
    dynamic_config = optuna_dynamic_hyperparameters(trial)
    config = {**static_config, **dynamic_config}
    # print(f"Trial {trial.number}: Configuration - {config}")

    kfold = KFold(n_splits=5, shuffle=True)
    aggregated_metrics = []

    best_objective_value = float('-inf')
    best_model_state = None

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        # print(f"Starting Fold {fold + 1}")
        train_loader, val_loader = get_data_loaders(dataset, train_idx, val_idx)
        model, optimizer = initialize_model(config)

        scheduler = StepLR(optimizer, step_size=trial.suggest_int("step_size", 5, 20), \
            gamma=trial.suggest_float("gamma", 0.1, 0.5))
        
        early_stopping = EarlyStopping(patience=10, delta=0.001)  # Early stopping instance


        for epoch in range(num_epochs):
            _, _ = train_one_epoch(model, train_loader)

            scheduler.step()
            win_rate, average_attempts, win_rate_by_length = simulate_games(
                    model=model,
                    word_list=None,  # Not using a word list in this scenario
                    char_to_idx=char_to_idx, 
                    idx_to_char=idx_to_char,
                    char_frequency=char_frequency, 
                    max_word_length=max_word_length, 
                    device=device, 
                    val_loader=val_loader,  # Pass the validation DataLoader
                    max_games_per_epoch=N_GAMES_PER_EPOCH,  # Number of games for validation
                    normalize=True, 
                    max_attempts=MAX_ATTEMPTS
            
            )

    #         win_rate, average_attempts, win_rate_by_length = validate_one_epoch(model, \
    #             val_loader, char_to_idx, idx_to_char, char_frequency, max_word_length, device)

            aggregated_metrics.append((win_rate, average_attempts, win_rate_by_length))

            objective_value = calculate_objective_value(win_rate, \
                average_attempts, win_rate_by_length)
            
            trial.report(objective_value, epoch)

            if objective_value > best_objective_value:
                best_objective_value = objective_value
                best_model_state = model.state_dict()  # Update best model state

            # Check early stopping
            if early_stopping(objective_value):
                break  # Stop the current fold if early stopping criteria are met

            # Pruning check
            if trial.should_prune():
                return process_aggregated_metrics(aggregated_metrics)

    # Save the best model at the end of the trial
    if best_model_state:
        model.load_state_dict(best_model_state)
        model.save_model()  # Save the model using your custom method

    final_objective_value = process_aggregated_metrics(aggregated_metrics)
    
    return final_objective_value


def calculate_objective_value(win_rate_percentage, avg_attempts_per_game, \
    win_rate_by_word_length):
    # Define weights for each metric
    weight_for_win_rate = 0.7  # Higher weight for win rate
    weight_for_average_attempts = 0.3  # Lower weight for average attempts

    # Normalize win rate (converting percentage to a range of 0 to 1)
    normalized_win_rate = win_rate_percentage / 100

    # Normalize average attempts (assuming max attempts per game is known)
    max_attempts_per_game = 6  # Example: maximum number of attempts allowed
    normalized_avg_attempts = avg_attempts_per_game / max_attempts_per_game

    # Normalize win rate by word length
    normalized_win_rate_by_length = normalize_win_rate_by_length(win_rate_by_word_length)

    # Combine metrics into a single objective value
    objective_value = (weight_for_win_rate * normalized_win_rate) - \
                      (weight_for_average_attempts * normalized_avg_attempts) + \
                      normalized_win_rate_by_length
    return objective_value

def normalize_win_rate_by_length(win_rate_by_length_dict):
    # Example logic for normalization
    max_possible_win_rate = 1  # Maximum win rate (100% as a decimal)
    normalized_scores = {length: win_rate / max_possible_win_rate for \
        length, win_rate in win_rate_by_length_dict.items()}
    # Average the normalized win rates across all word lengths
    average_normalized_score = sum(normalized_scores.values()) / len(normalized_scores)
    return average_normalized_score

def process_aggregated_metrics(aggregated_metrics):
    # Initialize sums for each metric
    sum_win_rate = 0
    sum_average_attempts = 0
    sum_win_rate_by_length = {}

    # Process each set of metrics
    for win_rate, average_attempts, win_rate_by_length in aggregated_metrics:
        sum_win_rate += win_rate
        sum_average_attempts += average_attempts
        for length, rate in win_rate_by_length.items():
            sum_win_rate_by_length[length] = sum_win_rate_by_length.get(length, 0) + rate

    # Calculate averages
    num_entries = len(aggregated_metrics)
    avg_win_rate = sum_win_rate / num_entries
    avg_average_attempts = sum_average_attempts / num_entries
    avg_win_rate_by_length = {length: rate / num_entries for length, \
        rate in sum_win_rate_by_length.items()}

    # Now calculate the final objective value using these averages
    final_objective_value = calculate_objective_value(avg_win_rate, \
        avg_average_attempts, avg_win_rate_by_length)
    return final_objective_value


# Utility functions (for cleaner code)
def get_data_loaders(dataset, train_idx, val_idx):
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, \
        shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_subset, batch_size=32, \
        shuffle=False, collate_fn=collate_fn)
    return train_loader, val_loader

def initialize_model(config):
    model = RNN(config)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return model, optimizer

# Utility functions
def optuna_dynamic_hyperparameters(trial):
    # Define and return dynamic hyperparameters based on the trial
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 150])
    output_mid_features = trial.suggest_categorical('output_mid_features', [50, 100, 200])
    miss_linear_dim = trial.suggest_categorical('miss_linear_dim', [50, 100, 150])
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    num_layers = trial.suggest_int('num_layers', 1, 3)

    return {
        'lr': lr,
        'hidden_dim': hidden_dim,
        'embedding_dim': embedding_dim,
        'output_mid_features': output_mid_features,
        'miss_linear_dim': miss_linear_dim,
        'dropout': dropout,
        'num_layers': num_layers
    }

In [ ]:
from pathlib import Path

# Define your models directory path
models_dir = Path('models/notebook_2')

# Create the directory if it does not exist
models_dir.mkdir(parents=True, exist_ok=True)

# # Assuming missed_chars_tensor is a list of tensors for missed characters
# dataset = HangmanDataset(features, \
#     labels, missed_chars, original_words)

# Static configuration
static_config = {
    'rnn': 'LSTM',
    'vocab_size': 27,  # 26 English alphabets + 1 (e.g., for underscore)
    'use_embedding': True,  # Typically a design choice
    'input_feature_size': 5,  # Based on your feature engineering strategy
    'models': str(models_dir)
}

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define num_epochs for training in each fold
num_epochs = 4 # Adjust as needed
N_TRIAL = 3

# ====================================================== #
# Define the direction for the objective: 'maximize' or 'minimize'
direction = 'maximize'  # Adjust based on your objective function

study = optuna.create_study(direction=direction)
study.optimize(lambda trial: objective(trial, dataset, \
    static_config, num_epochs), n_trials=N_TRIAL)
# ====================================================== #

In [ ]:
best_params = study.best_params
# Output the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"Value: {trial.value}")
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
best_trial = study.best_trial
print(f"Best trial number: {best_trial.number}")

# Assuming model saving includes the trial number in the filename
best_model_filename = f"models/trial_{best_trial.number+1}/LSTM.pth"
best_model = RNN.load_model(RNN, best_model_filename, device)

In [ ]:
STOP

In [ ]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

config

In [ ]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

# Initialize the RNN model with the combined configuration
model = RNN(config)

model = model.to(device)

# model.load_model(config['LSTM'], 2, 256, trial_number=10)

In [ ]:
STOP

##### Testing on unknown data

In [ ]:
%%capture

from scr.game import predict_next_character, simulate_game
from scr.model import RNN
import random
# random.seed(400)
# Your existing code for initializing the model, etc.

import random

def play_multiple_games(model, num_games, word_list, \
    char_to_idx, idx_to_char, char_frequency, max_word_length, device):
    game_results = []
    
    sampled_words = random.sample(word_list, num_games)  # Select unique words

    for random_word in sampled_words:
        with torch.no_grad():
            won, final_word, attempts_used = simulate_game(
                model, 
                random_word, 
                char_to_idx, 
                idx_to_char, 
                char_frequency, 
                max_word_length, 
                device, 
                normalize=True, 
                max_attempts=6
            )
        game_results.append((won, final_word, attempts_used))

    return game_results

# Example usage
num_games = 10**3
results = play_multiple_games(model, num_games, \
    word_list, char_to_idx, idx_to_char, \
        char_frequency, max_word_length, device)

# Analyzing results
total_wins = sum(result[0] for result in results)
win_rate = (total_wins / num_games) * 100
